In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,RidgeClassifier,RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier,BaggingClassifier,RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier,XGBRFClassifier
from catboost import CatBoostClassifier,CatBoost
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
np.random.seed(42)

In [2]:
data = pd.read_csv('./train.csv')

In [3]:
data.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [4]:
X = data.drop('loss',axis=1)
y = data['loss']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.375)

In [5]:
pres = [StandardScaler(),RobustScaler(),MinMaxScaler(),MaxAbsScaler(),Normalizer(),LabelEncoder()]

In [6]:
label_encoder = False

In [7]:
import wandb
PROJECT_NAME = 'Tabular-Aug-Clf'

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
model = GradientBoostingClassifier()
model.fit(X_train,y_train)
wandb.log({'Acc':model.score(X_train,y_train)})
wandb.log({'Val Acc':model.score(X_test,y_test)})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
wandb.init(project=PROJECT_NAME,name='Without Fe')
model = GradientBoostingClassifier()
model.fit(X_train,y_train)
wandb.log({'Acc':model.score(X_train,y_train)})
wandb.log({'Val Acc':model.score(X_test,y_test)})
wandb.finish()
for col in list(data.drop(['id','loss'],axis=1).columns):
    max_num = data[col].quantile(0.99)
    min_num = data[col].quantile(0.05)
    data = data[data[col] < max_num]
    data = data[data[col] > min_num]
wandb.init(project=PROJECT_NAME,name='With Fe')
model = GradientBoostingClassifier()
model.fit(X_train,y_train)
wandb.log({'Acc':model.score(X_train,y_train)})
wandb.log({'Val Acc':model.score(X_test,y_test)})
wandb.finish()

In [ ]:
for pre in pres:
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.375)
    if label_encoder is True:
        le = LabelEncoder()
        le.fit(y_train)
        y_train = le.transform(y_train)
    wandb.init(project=PROJECT_NAME,name=f'preproccessing-{pre}-labelencoder-{label_encoder}')
    transformation = pre
    transformation.fit(X_train)
    X_train = transformation.transform(X_train)
    model = GradientBoostingClassifier()
    print('Fitting')
    model.fit(X_train,y_train)
    print('Scoring Train')
    acc = model.score(X_train,y_train)
    wandb.log({'Acc':acc})
    print('Scoring Test')
    acc = model.score(transformation.transform(X_test),y_test)
    wandb.log({'Val Acc':acc})

wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Fitting
